In [26]:
from strategy import loadData, generate_signals, calculate_technical_indicators, doTrading
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [35]:
# data = loadData()
data = pd.read_csv('EURUSD5.csv', delimiter='\t', header=None, names=['timeStamp', 'open', 'high', 'low', 'close', 'volume'])
data = data[['timeStamp', 'close']]
data.rename(columns={'close':'mid'}, inplace=True)
data = data.tail(287*30)
data

,timeStamp,mid
91390,2024-06-04 19:10,1.08800
91391,2024-06-04 19:15,1.08812
91392,2024-06-04 19:20,1.08818
91393,2024-06-04 19:25,1.08809
91394,2024-06-04 19:30,1.08805
...,...,...
99995,2024-07-16 16:35,1.08846
99996,2024-07-16 16:40,1.08837
99997,2024-07-16 16:45,1.08855
99998,2024-07-16 16:50,1.08874


In [36]:
richData = calculate_technical_indicators(data.tail(287))
richData

,timeStamp,mid,SMA_short,SMA_long,RSI,EMA_short,EMA_long,MACD,Signal_line
99713,2024-07-15 17:05,1.09035,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99714,2024-07-15 17:10,1.09006,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99715,2024-07-15 17:15,1.08997,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99716,2024-07-15 17:20,1.09011,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99717,2024-07-15 17:25,1.08980,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
99995,2024-07-16 16:35,1.08846,1.088370,1.088058,57.485030,1.088330,1.088275,0.000055,0.000050
99996,2024-07-16 16:40,1.08837,1.088360,1.088042,53.216374,1.088336,1.088282,0.000054,0.000051
99997,2024-07-16 16:45,1.08855,1.088371,1.088043,52.095808,1.088369,1.088302,0.000067,0.000054
99998,2024-07-16 16:50,1.08874,1.088399,1.088051,59.217877,1.088426,1.088334,0.000092,0.000062


In [37]:
is_signal_col = lambda name: 'mid' in name or 'quote' in name or 'SMA' in name or 'RSI' in name or 'MACD' in name or 'Signal' in name
fig = px.line(richData, x = 'timeStamp', y = [col for col in richData.columns if is_signal_col(col)])
fig.show()

In [38]:
d = generate_signals(data)
d['signal'] = d['signal'].astype(str)

In [39]:
set(d['signal'].to_list())

{'-1', '-2', '0', '1', '2'}

In [40]:
is_signal_col = lambda name: 'mid' in name
line = px.line(d, x = 'timeStamp', y = [col for col in d.columns if is_signal_col(col)])
scatter = px.scatter(d, x = 'timeStamp', y='mid', color='signal', size='mid' )
plot = go.Figure(data=line.data + scatter.data)
plot.show()

In [33]:
# signal = (data['SMA_10'] > data['SMA_50']).to_numpy() * 1
nd = richData[['timeStamp', 'mid', 'MACD', 'Signal_line']]
nd['signal'] = 0
nd['macd_signal'] = None
nd.loc[(nd['MACD'] > nd['Signal_line']), 'signal'] = 1
nd.loc[(nd['MACD'] < nd['Signal_line']), 'signal'] = 0
nd['signal'] = nd['signal'].diff()
nd.loc[(nd['signal'] == -1), 'macd_signal'] = 0
nd.loc[(nd['signal'] == 1), 'macd_signal'] = 1
nd['macd_signal'] = nd['macd_signal'].astype('str')
# nd['sma_signal'] = signal
# nd.loc[(nd['sma_signal'] == 0)]
nd.drop(columns=['signal'], inplace=True)

nd

,timeStamp,mid,MACD,Signal_line,macd_signal
99713,2024-07-15 17:05,1.09035,NaN,NaN,None
99714,2024-07-15 17:10,1.09006,NaN,NaN,None
99715,2024-07-15 17:15,1.08997,NaN,NaN,None
99716,2024-07-15 17:20,1.09011,NaN,NaN,None
99717,2024-07-15 17:25,1.08980,NaN,NaN,None
...,...,...,...,...,...
99995,2024-07-16 16:35,1.08846,0.000055,0.000050,1
99996,2024-07-16 16:40,1.08837,0.000054,0.000051,None
99997,2024-07-16 16:45,1.08855,0.000067,0.000054,None
99998,2024-07-16 16:50,1.08874,0.000092,0.000062,None


In [34]:
is_signal_col = lambda name: 'mid' in name or 'quote' in name or 'SMA' in name or 'RSI' in name or 'MACD' in name or 'Signal' in name
line = px.line(nd, x = 'timeStamp', y = [col for col in nd.columns if is_signal_col(col)])
scatter = px.scatter(nd, x = 'timeStamp', y='mid', color='macd_signal', size='mid' )
plot = go.Figure(data=line.data + scatter.data)
plot.show()